In [1]:
import tensorflow as tf
import tflearn
import numpy as np
import gensim

In [2]:
class LSTM:
    def __init__(self, vocabulary_size=200, num_nodes=128, batch_size=20, num_unrollings=2, name=""):
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        self.num_unrollings = num_unrollings
        self.batch_size = batch_size
        self.name = name
        
        with tf.variable_scope("{}_LSTM_var".format(self.name) ,reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, vocabulary_size], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [vocabulary_size], tf.float32,
                                        tf.constant_initializer(0))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False)
            '''self.saved_state_backward = tf.get_variable("saved_state_backward", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False, name="backward_state")'''
                      
        
        self.train_data = list()
        for _ in range(self.num_unrollings + 1):
            self.train_data.append(
                tf.placeholder(tf.float32, shape=[self.batch_size, self.vocabulary_size]))
        self.train_inputs = self.train_data[:self.num_unrollings]
        self.train_labels = self.train_data[1:]
        
        self.outputs = list()
        self.output = self.saved_output
        self.state = self.saved_state
        
        for i in self.train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(self.output)
        
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
    
    def lstm_cell(self, inputs, hidden_layer, state): 
        
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state   
    
    def lstm_process(self, inputs, hidden_layer, state):
        for i in train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(output)

        return outputs
        '''with tf.control_dependencies([self.saved_output.assign(output),
                                     self.saved_state.assign(state)]):
            
            logits = tf.matmul(tf.concat(0, outputs), self.w) + self.b
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(tf.concat(0, logits), 
                                                        tf.concat(0, train_labels), name="loss"))
            
            global_step = tf.Variable(0)
            learning_rate = tf.train.exponential_decay(
                10.0, global_step, 5000, 0.1, staircase=True)
            optimizer = tf.train.RMSPropOptimizer(learning_rate)
            gradient, v = zip(*optimizer.compute_gradients(loss))
            
            
            tf.add_to_collection("loss", loss)
            tf.add_to_collection("logits", logits)
            tf.add_to_collection("train_labels", train_labels)
            
        '''
        #It seems we do not need those lines above
        
    def train(self):
        pass
    
    def __call__(self, inputs):
        self.lstm_process(inputs)
        

In [3]:
class attention:
    def __init__(self, h, s, name=""):
        '''
        Attention mechanism.
        arguments:
            h: [2*Hidden_size, Batch_size], encoder's hidden states
            s: [2*Hidden_size, Batch_size], decoder's hidden states
        '''
        self.name = name
        self.attn_logits = list()
        self.a = list()
        with tf.variable_scope("{}_attention_model".format(self.name)):
            self.Ua = tf.get_variable("Ua", [int(h[0].get_shape()[1])/2, int(h[0].get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            self.Wa = tf.get_variable("Wa", [int(s.get_shape()[1]), int(s.get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            #the following bias does not showed in original paper
            self.Ba = tf.get_variable("Ba", [1, int(s.get_shape()[1])])
            self.Va = tf.get_variable("Va", [1, int(s.get_shape()[1])])
            
            for h_j in h:
                self.attn_mlp = tf.tanh(
                    tf.matmul(self.Wa, tf.transpose(s)) + tf.matmul(self.Ua, tf.transpose(h_j)))
                self.attn_logits.append(tf.reshape(
                        tf.matmul(self.Va, self.attn_mlp), shape=[1])[0])
                
            self.a = tf.reshape(tf.nn.softmax(self.attn_logits),
                                shape=[len(h)], name="attention_value")
        
    def get_attention():
        return self.a
            

In [4]:
L = LSTM(batch_size=1, num_unrollings=2, name="TEST")

In [5]:
word_model = gensim.models.Word2Vec.load("models/word2vec.model")

In [6]:
test_word1 = word_model["china"]
test_word2 = word_model["uk"]
test_word3 = word_model["america"]

In [7]:
test_sentence = [test_word1, test_word2, test_word3]

In [8]:
h = tf.Variable(tf.truncated_normal((1,128)))
state = tf.Variable(tf.truncated_normal((1,128)))
test_inputs = tf.placeholder(tf.float32, shape=[1, 200])

In [9]:
h_j = list()
for _ in range(10):
    h_j.append(tf.Variable(tf.truncated_normal((1, 256))))
#h_j = tf.Variable(tf.truncated_normal((1, 256)))
s_j = tf.Variable(tf.truncated_normal((1, 128)))

In [10]:
A = attention(h_j, s_j, "1224")

In [13]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    feed_dict = dict()
    '''for i in range(3):
        feed_dict[L.train_data[i]] = np.reshape(test_sentence[i],(1,200))'''
    out = sess.run(A.a)
    print(out)

[  2.08016340e-10   3.54893189e-07   1.41401344e-07   4.55663640e-09
   4.30091319e-07   8.06008700e-08   7.11929024e-05   2.65648216e-01
   7.34279573e-01   3.33723693e-09]


In [12]:
out[0].reshape([1])

array([-10.23085594], dtype=float32)